
# Reddit

Reddit automations, because I hate reddit, so I should interact with it in my own way instead of their way.

Because whether i like it or not it is an information authority.




## Auth



### Login



#### the code

reddit login?


In [ ]:

const getClient = importer.import('selenium client')
const { Builder, Browser, By, Key, until } = require('selenium-webdriver')

//const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
//const PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'brian-reddit.json');

async function enterReddit(driver) {
  console.log('Reddit: Sign in required');

  //var credentials = require(PASSWORDS_FILE)

  //let body = await driver.findElement(By.css('body'))
  await driver.wait(until.elementLocated(By.css('#post-submit-form')), 30000);
  
  let loginStill, loginStill2, loginStill3
  try {
    loginStill = await driver.findElement(By.css('#captcha-internal'))
  } catch (e) {}
  try {
    loginStill2 = await driver.findElement(By.css('input[name*="session_password"]'))
  } catch (e) {}
  try {
    loginStill3 = await driver.findElement(By.xpath('//*[contains(text(),"Check your notifications")]'))
  } catch (e) {}

  if(loginStill3) {
    await new Promise(resolve => setTimeout(resolve, 20000))
    try {
      loginStill = false
      loginStill2 = false
      loginStill3 = await driver.findElement(By.xpath('//*[contains(text(),"Check your notifications")]'))
    } catch (e) {}
  }

  if(loginStill || loginStill2 || loginStill3) {
    throw new Error('captcha')
  }
}

async function loginReddit(driver) {
  if(!driver) {
    driver = await getClient();
  }

  try {
    let url = await driver.getCurrentUrl()
    let loggedIn = url.indexOf('reddit') > -1 && url.indexOf('login') == -1

    debugger
    if(loggedIn) {
      //if(await driver.findElement(By.xpath('iframe.authentication-iframe'))) {
      //  await driver.frame((await driver.element('iframe.authentication-iframe')).value)
        await enterReddit(driver)
        //await frame()
      //}
    } else {
      await driver.get('https://www.reddit.com/submit')

      let loginLink, loginLink2
      try {
        loginLink = await driver.findElement(By.xpath('//auth-flow-link[text()[contains(.,"Forgot password?")]]'))
      } catch (e) {}
      try {
        loginLink2 = await driver.findElement(By.xpath('//auth-flow-link[text()[contains(.,"Sign up")]]'))
      } catch (e) {}

      if(loginLink || loginLink2) {
        await enterReddit(driver)
      }
    }
  } catch (e) {
    driver.quit() // avoid leaving sessions hanging around

    throw e
  }

  return driver
}

module.exports = loginReddit



## Functions



### Post

create a post on reddit


#### the code

reddit post?


In [ ]:
const loginReddit = importer.import('reddit login')
const getClient = importer.import('selenium client')
const getAllQuery = importer.import('selenium query')
const { By } = require('selenium-webdriver')


async function redditPost(driver, content, startPage) {
  let {llmPrompt} = await importer.import('create llm session')

  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  if(!startPage.includes('/submit')) {
    startPage += '/submit?type=TEXT'
  }

  if(!driver) {
    driver = await getClient()
  }

  await driver.get(startPage)

  await loginReddit(driver)

  await new Promise(resolve => setTimeout(resolve, 1500))

  let titleStr = (/\*\*(.*?)\*\*\n*/gi).exec(content)
  if(titleStr) {
    content = content.replace(titleStr[0], '')
    titleStr = titleStr[1].replaceAll(/[#*"']/gi, '')
  } else {
    titleStr = await llmPrompt(
      'Generate a short title for this summary:\n' + 
      content)
    titleStr = titleStr.replaceAll(/[#*"']/gi, '')
  }

  try {
    let title = await driver.findElement(By.css('faceplate-textarea-input[name*="title"]'))
    await title.click()
    //await driver.executeScript('arguments[0].click();', title)
    await driver.actions().sendKeys(titleStr).perform()

    let switchButton = await getAllQuery(driver, [
      'shreddit-composer',
      'button[aria-label*="Switch to Markdown"]'
    ])
    await driver.executeScript('arguments[0].click();', switchButton[0])
    await new Promise(resolve => setTimeout(resolve, 500))

    let body = await getAllQuery(driver, [
      'shreddit-composer',
      'shreddit-markdown-composer',
      'textarea[placeholder*="Body"]'
    ])
    await driver.executeScript('arguments[0].click();', body[0])
    await driver.actions().sendKeys(content).perform()

    let submit = await driver.findElement(By.css('#submit-post-button'))
    submit.click()
    await new Promise(resolve => setTimeout(resolve, 1500))

  } catch (e) {
    //driver.quit()

    throw e
  }

}

module.exports = redditPost



#### test reddit post?



In [ ]:

const redditPost = importer.import('reddit post')


async function testPost() {
  await redditPost(void 0, 'this is a test', 'CollapseGently')
}


module.exports = testPost



## Bot



### Weekly Summerizer



#### the code

posts articles from a reddit list to another reddit owned by me but with a bunch of llm conversion in between.

reddit weekly?


In [ ]:
const redditLinks = importer.import('reddit scraper')
const getClient = importer.import('selenium client')
const redditList = importer.import('reddit month of links')
const redditPost = importer.import('reddit post')
const summerizeArticle = importer.import('summarize llm article')
const {alternativeSummary, CONVERSION_PROMPTS} = importer.import('convert summaries')
const { persistSummaries } = importer.import('default link collector')
const extractArticle = importer.import('extract llm article')

// TODO: send an email or post updates on reddit.com/r/collapseGently?

async function redditWeekly(
  startPage = 'CollapseSupport+climatechange+collapse+economicCollapse',
  postPage = 'CollapseGently'
) {

  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  let driver = await getClient()

  // TODO: get top
  let top = await redditLinks(driver, startPage + '/top/')
  let topLinks = top.map(post => post.link)

  // TODO: sort by most comments
  let posts = await redditList(driver, startPage)
  let topCommented = posts.sort((a, b) => {
    let aInt = parseInt(a.comment.replace(/comments*/gi, '').trim())
    let bInt = parseInt(b.comment.replace(/comments*/gi, '').trim())
    return bInt - aInt
  }).filter(post => !topLinks.includes(post.link))

  let freshPosts = topCommented.slice(0, 25).concat(top.slice(0, 25))
  let summaries = persistSummaries()

  // TODO: loop through top 20 (10 of each) and repost
  for(let i = 0; i < freshPosts.length; i++) {
    let summary = summaries[freshPosts[i].link]
    if(!summary) {
      let article = await extractArticle(driver, freshPosts[i].link)
      summary = await summerizeArticle(article)
    }

    // TODO: extract funny summary instead
    let rand = Math.round(Math.random() * (CONVERSION_PROMPTS.length - 1)) + 1
    let alternative = await alternativeSummary(summary, CONVERSION_PROMPTS[rand])

    await redditPost(driver, 
      'TLDR: ' + alternative[1] 
      + '\n\n' + alternative[0] 
      + '\n\n' + '[' + freshPosts[i].link + '](' + freshPosts[i].link + ')\n', 
      postPage)
  }

}

module.exports = redditWeekly



## Utilities


### Week/Month Extractor

Extract and entire week/month of links, go to the next page until the articles are older than 1 week/month.



#### the code

reddit month of links?



In [ ]:

const redditLinks = importer.import('reddit scraper')
const getClient = importer.import('selenium client')

async function redditList(driver, startPage, timeSpan = 'week') {
  if(!startPage) {
    startPage = 'https://www.reddit.com/r/CollapseSupport+climatechange+collapse+economicCollapse/'
  }

  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  if(!driver)
    driver = await getClient()

  try {

    let safety = 20

    let weekAgo = new Date(Date.now() - 1000 * 60 * 60 * 24 * 7)
    let monthAgo = new Date(Date.now() - 1000 * 60 * 60 * 24 * 7 * 4.2)

    let finalResult = []

    let result
    let beforeTimeSpan = 0
    do {

      console.log(startPage)

      result = await redditLinks(driver, startPage)

      finalResult = finalResult.concat(result)

      startPage = result.next

      // not yet includes any articles over a month ago
      beforeTimeSpan = result.filter(r => timeSpan == 'month' 
        ? r.time < monthAgo : r.time < weekAgo).length

      safety--

      if(result.next && beforeTimeSpan == 0 && safety > 0) {
        await new Promise(resolve => setTimeout(resolve, 1000))
      }

    } while(result.next && beforeTimeSpan == 0 && safety > 0)

    return finalResult;

  } catch (e) {
    driver.quit()

    throw e
  }
}


module.exports = redditList





### Scrape Links



#### the code

reddit scraper?

TODO: go to reddit and download links and articles from every page in a list.


In [ ]:
const selectDom = importer.import('selenium select')
const getClient = importer.import('selenium client')
const {URL} = require('url')

async function redditLinks(driver, startPage) {
  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  let startUrl = new URL(startPage)

  if(!driver) {
    driver = await getClient()
  }

  try {
    await driver.get(startPage)

    await new Promise(resolve => setTimeout(resolve, 1000))

    let links = await selectDom(driver, [
      '//div[contains(@role, "main")]//div[contains(@class, "link") and not(contains(@class, "linklisting")) and not(contains(@class, "promoted"))]'])

    // TODO: get some special links, get comment count, titles, finally next page
    let results = []
    for(let i = 0; i < links.length; i++) {
      let result = await selectDom(driver, {
        title: './/a[contains(@class, "title")]/text()',
        link: './/a[contains(@class, "title")]/@href',
        time: './/time/@datetime',
        comment: './/a[contains(@class, "comments")]/text()',
      }, links[i])
      results.push(result)
    }

    let next = await selectDom(driver, '//a[contains(@rel, "next")]/@href')

    let objectArray = results.map(r => ({
      title: r.title,
      time: new Date(r.time),
      link: r.link.includes('://') 
        ? r.link : ((!r.link.startsWith('/') 
        ? (startUrl.origin + '/' + startUrl.pathname + './') : startUrl.origin) + r.link),
      comment: r.comment
    }))
    objectArray.next = next
    return objectArray
  } catch (e) {
    driver.quit()

    throw e
  }
}

module.exports = redditLinks


#### test scraper

test reddit scraper?

In [ ]:
const redditLinks = importer.import('reddit scraper')
const getClient = importer.import('selenium client')

async function testScraper(startPage = 'https://www.reddit.com/r/CollapseSupport+climatechange+collapse+economicCollapse/') {
  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  driver = await getClient()

  let result = await redditLinks(driver, startPage)

  driver.quit()

  return result
}


module.exports = testScraper

